In [1]:
import numpy as np
import pandas as pd
import csv
from dask.distributed import Client, progress
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

In [2]:
client = Client(processes=False, threads_per_worker=4,
                n_workers=4, memory_limit='8GB')

In [3]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://192.168.0.105:8787/status,
Dashboard: http://192.168.0.105:8787/status,Workers: 4
Total threads: 16,Total memory: 29.80 GiB
Status: running,Using processes: False
Comm: inproc://192.168.0.105/31932/1,Workers: 4
Dashboard: http://192.168.0.105:8787/status,Total threads: 16
Started: Just now,Total memory: 29.80 GiB
Comm: inproc://192.168.0.105/31932/4,Total threads: 4
Dashboard: http://192.168.0.105:49328/status,Memory: 7.45 GiB
Nanny: None,


In [4]:
df = dd.read_csv('data/train2.csv', dtype={"district_id": "Int64", "city_id": "Int64", "id": "Int64", "is_business": "Int64", "region_id": "Int64", "price": "Int64", "created_at_first": "str",
"params": "str"}, engine="python", encoding="utf-8")

In [5]:
districts = dd.read_csv("data/districts.csv", dtype={'id': 'Int64'})

In [6]:
cities = dd.read_csv("data/cities.csv", dtype={'id': 'Int64'})

In [7]:
data = dd.merge(df, cities, left_on="city_id", right_on="id")

In [8]:
data = data.drop("id_y", axis=1)

In [9]:
districts = districts.rename(columns={"id": "district_id"})

In [10]:
data2 = dd.merge(data, districts, on="district_id", how="left")

In [11]:
data = data2.drop(columns = {"Unnamed: 0", "id_x", "created_at_first", "region_id", "lon_x", "lat_x"})

In [12]:
data = data.rename(columns = {"name_pl_x": "city_name", "name_pl_y": "disctrict_name"})

In [13]:
data.head()

,category,is_business,district_id,city_id,params,price,city_name,disctrict_name,disctrict_name,lon_y,lat_y
0,Mieszkanie na sprzedaż,1,12867,210,price[currency]<=>PLN<br>m<=>72.14<br>rooms_nu...,389556,Olsztyn,Dajtki,Dajtki,20.42342,53.76516
1,Mieszkanie na sprzedaż,1,280,210,price[currency]<=>PLN<br>m<=>49.6<br>rooms_num...,222000,Olsztyn,Kormoran,Kormoran,20.49830,53.76785
2,Mieszkanie na sprzedaż,1,590,210,price[currency]<=>PLN<br>m<=>39.41<br>rooms_nu...,254195,Olsztyn,Nagórki,Nagórki,20.49327,53.75620
3,Dom na sprzedaż,1,2568,210,price[currency]<=>PLN<br>terrain_area<=>703<br...,690000,Olsztyn,Redykajny,Redykajny,20.43742,53.81064
4,Mieszkanie na sprzedaż,1,6168,210,price[currency]<=>PLN<br>m<=>36.53<br>rooms_nu...,209000,Olsztyn,Osiedle Mazurskie,Osiedle Mazurskie,20.51410,53.76086
